In [ ]:
# default_exp market-maker

# module issu de 01_market-maker

> module du 2ème notebook de test

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from nbdev_bot.interface import deribit_interface
import threading
import time
from nbdev_bot.config import api_ID, api_secret

In [ ]:
#export
### CONSTANTS ###
AMOUNT_TO_TRADE = 10
INSTRUMENT_NAME = 'BTC-PERPETUAL'
CLIENT_ID = api_ID
CLIENT_SECRET = api_secret
### ### ###

In [ ]:
#export
### system value ###
deribit = deribit_interface.Deribit(test=True, 
	client_ID=CLIENT_ID,
 	client_secret=CLIENT_SECRET)
logwritter = deribit.logwritter

buy_order_id, sell_order_id, bid_last, ask_last, case_pose = None, None, None, None, None
first_trade = True
step_in_algo = 0
### ### ###


[20201112,15:52:06] Auth OK
############

[20201112,15:52:06] public/auth


In [ ]:
#export
### functions ###

def First_setuper():
	global deribit, AMOUNT_TO_TRADE, INSTRUMENT_NAME, buy_order_id, sell_order_id, bid_last, ask_last
	ask_last = deribit.Orderbook['asks'][0][0]
	bid_last = deribit.Orderbook['bids'][0][0]
	logwritter('now: bid/ask %f/%f' % (bid_last, ask_last))

	buy_order_id = deribit.make_order(side='buy',
		instrument_name=INSTRUMENT_NAME,
		amount=AMOUNT_TO_TRADE,
		price=bid_last,
		post_only=True)['order']['order_id']
	sell_order_id = deribit.make_order(side='sell',
		instrument_name=INSTRUMENT_NAME,
		amount=AMOUNT_TO_TRADE,
		price=ask_last,
		post_only=True)['order']['order_id']
	logwritter('Setup first order OK')



def Order_manager(timeout=0.1):
	global deribit, AMOUNT_TO_TRADE, INSTRUMENT_NAME, buy_order_id, sell_order_id, step_in_algo, case_pose, ask_last, bid_last, first_trade
	while True:
		ask_now = deribit.Orderbook['asks'][0][0]
		bid_now = deribit.Orderbook['bids'][0][0]

		if not first_trade: # Change order to actual price
			if (not ask_now==ask_last) or (not bid_now==ask_last):
				logwritter('change order to actual price')
				deribit.edit_order(order_id=buy_order_id, amount=2*AMOUNT_TO_TRADE, price=bid_now)
				deribit.edit_order(order_id=sell_order_id, amount=2*AMOUNT_TO_TRADE, price=ask_now)
			ask_last = ask_now
			bid_last = bid_now

		if step_in_algo==0:
			logwritter('step 0')
			if case_pose=='buy_first_filled' or first_trade:
				status_buy = deribit.get_order_state(order_id=buy_order_id)['order_state']
				if status_buy=='filled':
					if first_trade:
						case_pose = 'buy_first_filled'
					price_editor_busy = False
					step_in_algo = 1
			if case_pose=='sell_first_filled' or first_trade:
				status_sell = deribit.get_order_state(order_id=sell_order_id)['order_state']
				if status_sell=='filled':
					if first_trade:
						case_pose = 'sell_first_filled'
					price_editor_busy = False
					step_in_algo = 1

		if step_in_algo==1:
			logwritter('step 1')
			if case_pose=='buy_first_filled' and not first_trade:
				sell_order_id = deribit.make_order(side='sell',
												instrument_name=INSTRUMENT_NAME,
												amount=2*AMOUNT_TO_TRADE,
												price=ask_now,
												post_only=True)['order']['order_id']
			if case_pose=='buy_first_filled' and first_trade:
				first_trade = False
				deribit.edit_order(order_id=sell_order_id, amount=2*AMOUNT_TO_TRADE, price=ask_now)
			if case_pose=='sell_first_filled' and not first_trade:
				buy_order_id = deribit.make_order(side='buy',
												instrument_name=INSTRUMENT_NAME,
												amount=2*AMOUNT_TO_TRADE,
												price=bid_now,
												post_only=True)['order']['order_id']
			if case_pose=='sell_first_filled' and first_trade:
				first_trade = False
				deribit.edit_order(order_id=buy_order_id, amount=2*AMOUNT_TO_TRADE, price=bid_now)
			step_in_algo = 2
			continue

		if step_in_algo==2:
			logwritter('step 2')
			status_order = None
			if case_pose=='buy_first_filled':
				status_order = deribit.get_order_state(order_id=sell_order_id)['order_state']
			if case_pose=='sell_first_filled':
				status_order = deribit.get_order_state(order_id=buy_order_id)['order_state']
			if status_order=='filled':
				if case_pose=='buy_first_filled':
					buy_order_id = deribit.make_order(side='buy',
								instrument_name=INSTRUMENT_NAME,
								amount=2*AMOUNT_TO_TRADE,
								price=bid_now,
								post_only=True)['order']['order_id']
				if case_pose=='sell_first_filled':
					sell_order_id = deribit.make_order(side='sell',
								instrument_name=INSTRUMENT_NAME,
								amount=2*AMOUNT_TO_TRADE,
								price=ask_now,
								post_only=True)['order']['order_id']
				step_in_algo = 0
		time.sleep(timeout)
### ### ###

In [ ]:
#export
if __name__=="__main__":
	#open('log.log', 'w')
	deribit.start_orderbook_update(instrument_name=INSTRUMENT_NAME)
	time.sleep(3)
	First_setuper()
	Order_manager()

--- request header ---
GET /ws/api/v2 HTTP/1.1
Upgrade: websocket
Connection: Upgrade
Host: test.deribit.com
Origin: http://test.deribit.com
Sec-WebSocket-Key: jgTxidxd2dJqhcTCMf6GWA==
Sec-WebSocket-Version: 13


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Server: nginx/1.17.9
Date: Thu, 12 Nov 2020 14:56:42 GMT
Connection: upgrade
Access-Control-Allow-Methods: GET, POST, OPTIONS
Access-Control-Allow-Headers: Authorization,User-Agent,Range,X-Requested-With,Content-Type,Partner
Cache-Control: no-store
Vary: Origin,Authorization,Partner
Access-Control-Allow-Origin: http://test.deribit.com
upgrade: websocket
sec-websocket-accept: lbzhUBafvs3rByAEYSw/NjY3vA0=
-----------------------
send: b'\x81\xf7\xca\x98\xe2\x81\xb1\xba\x88\xf2\xa5\xf6\x90\xf1\xa9\xba\xd8\xa1\xe8\xaa\xcc\xb1\xe8\xb4\xc2\xa3\xa7\xfd\x96\xe9\xa5\xfc\xc0\xbb\xea\xba\x92\xf4\xa8\xf4\x8b\xe2\xe5\xeb\x97\xe3\xb9\xfb\x90\xe8\xa8\xfd\xc0\xad\xea\xba\x8b\xe5\xe8\xa2\xc2\xb5\xf8\xb4\xc2\xa3\xb

{"jsonrpc":"2.0","id":42,"result":["book.BTC-PERPETUAL.none.1.100ms"],"usIn":1605193002106184,"usOut":1605193002106482,"usDiff":298,"testnet":true}
{"jsonrpc":"2.0","method":"subscription","params":{"channel":"book.BTC-PERPETUAL.none.1.100ms","data":{"timestamp":1605175242436,"instrument_name":"BTC-PERPETUAL","change_id":4679359005,"bids":[[16127.5,14600.0]],"asks":[[16133.5,46640.0]]}}}
{"jsonrpc":"2.0","method":"subscription","params":{"channel":"book.BTC-PERPETUAL.none.1.100ms","data":{"timestamp":1605193002241,"instrument_name":"BTC-PERPETUAL","change_id":4680200660,"bids":[[16021.5,1790.0]],"asks":[[16027.5,16040.0]]}}}
{"jsonrpc":"2.0","method":"subscription","params":{"channel":"book.BTC-PERPETUAL.none.1.100ms","data":{"timestamp":1605193002708,"instrument_name":"BTC-PERPETUAL","change_id":4680200703,"bids":[[16021.5,1790.0]],"asks":[[16030.0,16040.0]]}}}
{"jsonrpc":"2.0","method":"subscription","params":{"channel":"book.BTC-PERPETUAL.none.1.100ms","data":{"timestamp":1605193002

send: b'\x81\xfe\x01\x08\xab\xefh\x1a\xd0\xcd\x02i\xc4\x81\x1aj\xc8\xcdR:\x89\xddF*\x89\xc3H8\xc2\x8bJ \x8b\xdaZ-\x9e\xc3H8\xc6\x8a\x1cr\xc4\x8bJ \x8b\xcd\x18h\xc2\x99\tn\xce\xc0\no\xd2\xcdD:\x89\x9f\th\xca\x82\x1b8\x91\xcf\x138\xc2\x81\x1bn\xd9\x9a\x05\x7f\xc5\x9b7t\xca\x82\r8\x91\xcfJX\xff\xacEJ\xee\xbd8_\xff\xba)V\x89\xc3H8\xca\x82\x07o\xc5\x9bJ \x8b\xdeX6\x8b\xcd\x1cc\xdb\x8aJ \x8b\xcd\x04s\xc6\x86\x1c8\x87\xcfJv\xca\x8d\rv\x89\xd5Ht\xde\x83\x046\x8b\xcd\x18h\xc2\x8c\r8\x91\xcfY,\x9b\xddY4\x9e\xc3H8\xdf\x86\x05\x7f\xf4\x86\x06E\xcd\x80\x1ay\xce\xcdR:\x89\x88\x07u\xcf\xb0\x1cs\xc7\xb0\x0b{\xc5\x8c\rv\xc7\x8a\x0c8\x87\xcfJj\xc4\x9c\x1cE\xc4\x81\x04c\x89\xd5Hn\xd9\x9a\r6\x8b\xcd\x1a\x7f\xcf\x9a\x0b\x7f\xf4\x80\x06v\xd2\xcdR:\xcd\x8e\x04i\xce\xc3H8\xdf\x9d\x01}\xcc\x8a\x1a8\x91\xcf\x06o\xc7\x83\x15g'
send: b'\x81\xfe\x01\t+\xaa\xcc,P\x88\xa6_D\xc4\xbe\\H\x88\xf6\x0c\t\x98\xe2\x1c\t\x86\xec\x0eB\xce\xee\x16\x0b\x9f\xfe\x1b\x1e\x86\xec\x0eF\xcf\xb8DD\xce\xee\x16\x0b\x88\xbc^B\xdc\xadXN\x

{"jsonrpc":"2.0","method":"subscription","params":{"channel":"book.BTC-PERPETUAL.none.1.100ms","data":{"timestamp":1605193004391,"instrument_name":"BTC-PERPETUAL","change_id":4680200855,"bids":[[16021.5,1790.0]],"asks":[[16031.5,16040.0]]}}}

[20201112,15:56:44] now: bid/ask 16021.500000/16031.500000

[20201112,15:56:44] private/buy

[20201112,15:56:44] private/sell

[20201112,15:56:45] Setup first order OK

[20201112,15:56:45] step 0

[20201112,15:56:45] private/get_order_state

[20201112,15:56:45] private/get_order_state


send: b'\x81\xe9\x96\x95\xe4g\xed\xb7\x8e\x14\xf9\xfb\x96\x17\xf5\xb7\xdeG\xb4\xa7\xcaW\xb4\xb9\xc4E\xff\xf1\xc6]\xb6\xa0\xd6P\xa3\xb9\xc4E\xfb\xf0\x90\x0f\xf9\xf1\xc6]\xb6\xb7\x94\x15\xff\xe3\x85\x13\xf3\xba\x83\x02\xe2\xca\x8b\x15\xf2\xf0\x968\xe5\xe1\x85\x13\xf3\xb7\xc8G\xb4\xe5\x85\x15\xf7\xf8\x97E\xac\xb5\x9fE\xf9\xe7\x80\x02\xe4\xca\x8d\x03\xb4\xaf\xc4E\xa2\xa3\xdcW\xa4\xa5\xd4_\xaf\xad\xc6\x1a\xeb'
send: b"\x81\xe9:\x05\x8c|A'\xe6\x0fUk\xfe\x0cY'\xb6\\\x187\xa2L\x18)\xac^Sa\xaeF\x1a0\xbeK\x0f)\xac^W`\xf8\x14Ua\xaeF\x1a'\xfc\x0eSs\xed\x08_*\xeb\x19NZ\xe3\x0e^`\xfe#Iq\xed\x08_'\xa0\\\x18u\xed\x0e[h\xff^\x00%\xf7^Uw\xe8\x19HZ\xe5\x18\x18?\xac^\x0e3\xb4L\x085\xbcE\n6\xae\x01G"
send: b'\x81\xe9i\xe5Z\xdf\x12\xc70\xac\x06\x8b(\xaf\n\xc7`\xffK\xd7t\xefK\xc9z\xfd\x00\x81x\xe5I\xd0h\xe8\\\xc9z\xfd\x04\x80.\xb7\x06\x81x\xe5I\xc7*\xad\x00\x93;\xab\x0c\xca=\xba\x1d\xba5\xad\r\x80(\x80\x1a\x91;\xab\x0c\xc7v\xffK\x95;\xad\x08\x88)\xfdS\xc5!\xfd\x06\x97>\xba\x1b\xba3\xbbK\xdfz\xfd]\xd3b\xef[\x

...

send: b'\x81\xe92\n\xb4(I(\xde[]d\xc6XQ(\x8e\x08\x108\x9a\x18\x10&\x94\n[n\x96\x12\x12?\x86\x1f\x07&\x94\n_o\xc0@]n\x96\x12\x12(\xc4Z[|\xd5\\W%\xd3MFU\xdbZVo\xc6wA~\xd5\\W(\x98\x08\x10z\xd5ZSg\xc7\n\x08*\xcf\n]x\xd0M@U\xddL\x100\x94\n\x06<\x8c\x18\x00:\x87\x1b\n<\x96UO'
send: b"\x81\xfe\x00~.aF\xdfUC,\xacA\x0f4\xafMC|\xff\x0cSh\xef\x0cMf\xfdG\x05d\xe5\x0eTt\xe8\x1bMf\xfdC\x042\xb7A\x05d\xe5\x0eC6\xadG\x17'\xabKN#\xbbG\x15d\xf3\x0eC6\xbe\\\x00+\xac\x0c[f\xa4\x0c\x0e4\xbbK\x13\x19\xb6JC|\xff\x0cUp\xe7\x1eSv\xec\x1dYp\xfd\x02Ad\xbeC\x0e3\xb1ZC|\xff\x1cQj\xff\x0c\x114\xb6M\x04d\xe5\x0ePp\xef\x1fXh\xefS\x1c"
send: b"\x81\xfe\x00~UKX\xa9.i2\xda:%*\xd96ib\x89wyv\x99wgx\x8b</z\x93u~j\x9e`gx\x8b8.,\xc1:/z\x93ui(\xdb<=9\xdd0d=\xcd<?z\x85ui(\xc8'*5\xdawqx\xd2w$*\xcd09\x07\xc01ib\x89w\x7fn\x91eyh\x9bdsa\x8bykz\xc88$-\xc7!ib\x89g{t\x89w;*\xc06.z\x93uzn\x99gzv\x99(6"
send: b"\x81\xe9\x86\x1c\x17W\xfd>}$\xe9re'\xe5>-w\xa4.9g\xa407u\xefx5m\xa6)%`\xb307u\xebyc?\xe9x5m\xa6>g%\xefjv#\xe33p2\xf2Cx%\xe2ye\


[20201112,15:57:34] {'jsonrpc': '2.0', 'id': 5275, 'error': {'message': 'not_open_order', 'code': 11044}, 'usIn': 1605193054191215, 'usOut': 1605193054192290, 'usDiff': 1075, 'testnet': True}

[20201112,15:57:34] _sender error: 'result'

[20201112,15:57:34] step 2

[20201112,15:57:34] private/get_order_state
{"jsonrpc":"2.0","method":"subscription","params":{"channel":"book.BTC-PERPETUAL.none.1.100ms","data":{"timestamp":1605193054272,"instrument_name":"BTC-PERPETUAL","change_id":4680203882,"bids":[[16019.0,19520.0]],"asks":[[16021.0,16020.0]]}}}

[20201112,15:57:34] change order to actual price

[20201112,15:57:34] private/edit

[20201112,15:57:34] private/edit

[20201112,15:57:34] {'jsonrpc': '2.0', 'id': 5275, 'error': {'message': 'not_open_order', 'code': 11044}, 'usIn': 1605193054386900, 'usOut': 1605193054388074, 'usDiff': 1174, 'testnet': True}

[20201112,15:57:34] _sender error: 'result'

[20201112,15:57:34] step 2

[20201112,15:57:34] private/get_order_state


send: b'\x81\xfe\x00~\x18\x0f\xb3\xcac-\xd9\xb9wa\xc1\xba{-\x89\xea:=\x9d\xfa:#\x93\xe8qk\x91\xf08:\x81\xfd-#\x93\xe8uj\xc7\xa2wk\x91\xf08-\xc3\xb8qy\xd2\xbe} \xd6\xaeq{\x91\xe68-\xc3\xabjn\xde\xb9:5\x93\xb1:`\xc1\xae}}\xec\xa3|-\x89\xea:;\x85\xf2(=\x83\xf9+7\x85\xe84/\x91\xabu`\xc6\xa4l-\x89\xea*?\x9f\xea:\x7f\xc1\xa3{j\x91\xf08>\x85\xfa)6\x9d\xfaer'
send: b'\x81\xfe\x00~EO\xc2\xa4>m\xa8\xd7*!\xb0\xd4&m\xf8\x84g}\xec\x94gc\xe2\x86,+\xe0\x9eez\xf0\x93pc\xe2\x86(*\xb6\xcc*+\xe0\x9eem\xb2\xd6,9\xa3\xd0 `\xa7\xc0,;\xe0\x88em\xb2\xc57.\xaf\xd7gu\xe2\xdfg \xb0\xc0 =\x9d\xcd!m\xf8\x84g{\xf4\x9cu}\xf2\x96tw\xfb\x86io\xe0\xc5( \xb7\xca1m\xf8\x84w\x7f\xee\x84g?\xb0\xcd&*\xe0\x9ee~\xf4\x94w~\xec\x9482'
send: b'\x81\xe9\x9b\xe2\x1c\xb8\xe0\xc0v\xcb\xf4\x8cn\xc8\xf8\xc0&\x98\xb9\xd02\x88\xb9\xce<\x9a\xf2\x86>\x82\xbb\xd7.\x8f\xae\xce<\x9a\xf6\x87h\xd0\xf4\x86>\x82\xbb\xc0l\xca\xf2\x94}\xcc\xfe\xcd{\xdd\xef\xbds\xca\xff\x87n\xe7\xe8\x96}\xcc\xfe\xc00\x98\xb9\x92}\xca\xfa\x8fo\x9a\xa1\xc2g\x9a\xf4\x


[20201112,15:57:34] change order to actual price

[20201112,15:57:34] private/edit

[20201112,15:57:34] private/edit
{"jsonrpc":"2.0","method":"subscription","params":{"channel":"book.BTC-PERPETUAL.none.1.100ms","data":{"timestamp":1605193054470,"instrument_name":"BTC-PERPETUAL","change_id":4680203903,"bids":[[16019.0,19520.0]],"asks":[[16027.5,16020.0]]}}}

[20201112,15:57:34] {'jsonrpc': '2.0', 'id': 5275, 'error': {'message': 'not_open_order', 'code': 11044}, 'usIn': 1605193054561136, 'usOut': 1605193054562014, 'usDiff': 878, 'testnet': True}

[20201112,15:57:34] _sender error: 'result'

[20201112,15:57:34] step 2

[20201112,15:57:34] private/get_order_state


KeyboardInterrupt: 

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_deribit_interface.ipynb.
Converted 01_market-maker.ipynb.
Converted index.ipynb.
